##  Inspired by Chang
## ETF featuress, using atr_std= 1.5* ATR - 2.0 std to see whether it is in a trend folloiwng or mean reversion period
#
### a. 如果這值很大，代表現在是盤整，可以等第一根突破去追趨勢.
### b. 如果盤整越久，突破後成為趨勢的機率越高

##
##


### a.如果今天股價跟昨天相比沒有劇烈漲跌，那麼今天的波動範圍就是今日最高價減去今日最低價；
##

### b.如果今天大盤開盤是跳空上漲，而且這個空沒有被回補，那麼如果還是以今高減今低當作今天價格的波動幅度，
### 那顯然就遺漏了跳空上漲的部份，所以要將那部份加回來，所以今天TR就是今日最高價減去昨日收盤價；取絕對值
##
##
### c.如果今天大盤開盤是跳空下跌，而且這個空沒有被回補，那麼如果還是以今高減今低當作今天價格的波動幅度，
### 那顯然就遺漏了跳空下跌的部份，所以要將那部份加回來，所以今天TR就是今日最低價減去昨日收盤價的取絕對值
##
##
### calculate True Range (TR)
### (definition TR: max value of (high-low, abs(high-prev_close), abs(low-prev_close))
### 3. TRMA = n_days rolling average of TR (in here, n_days =20)

### 4. ATR = smooth mean(KD like) of TRMA : ATR(t)  = 2/3*ATR(t-1) + 1/3 * (TRMA(t));  ATR(0) = TRMA(0)

### 5. std_20 = rolling standadrd deviation of closing price
### 6. chang_idx: 1.5*ATR - 2* std_20

In [1]:
# copy from ryan's tech.py 
def _smoothing(seq, w, initial, name=None):
    """Smoothing calculation, such as k-index, d-index and ema.
    """
    y = [initial] * len(seq)
    wb = 1 - w
    for i, v in enumerate(seq[1:], 1):
        y[i] = w * v + wb * y[i - 1]
    out = pd.Series(y)
    if name:
        out.name = name
    return out


In [2]:
import numpy as np
import pandas as pd
from datetime import datetime


def _atr_std (price, high=None, low=None, close=None, w=0.33,n_days=20):
    """Calculate the Raw Stochastic Value.
    TODO:


    Params:
        price: type: pd.Series, close price
        high: type: pd.Series, high of the day
        low: type: pd.Series, low of the day
        n_days: day period for calculation, type: int

    Return: ATR, std_20, chang_idx
    """

    previous = price.shift(1).fillna(price[0])  # previous day close, fillna as first day
    tr = pd.concat([high - low, (high - previous).abs(), (low-previous).abs()], axis=1).max(axis=1) # True Range, defined as above
    trma =  tr.rolling(n_days).mean().fillna(tr[:n_days].mean())  # rolling tr, fillna with first value of trma
    std_20 =price.rolling(n_days).std().fillna(price[:n_days].std())  # rolling stddev20, fill nan with first value of std_20
    std_20.name = 'std_20'
    atr = _smoothing(trma, w=0.33,initial=trma[0], name='atr')
    chang_idx = 1.5*atr - 2.0*std_20
    chang_idx.name = 'atr_std20'
    
    result = pd.concat([chang_idx, atr, std_20], axis=1)

    return chang_idx, atr, std_20
    #return result



## toy example 

In [3]:

test_dict={'A':[1,1,3,2,6,2,4],
           'B': [2,4,5,2,7,3,1],
            'C':[3,5,6,2,1,4,5],
            'D':[5,1,4,3,2,1,3]}
test_df = pd.DataFrame(test_dict)

In [4]:
result = _atr_std(test_df['A'], high=test_df['B'], low= test_df['C'], n_days=3)
result = pd.concat(result, axis=1)
result

,atr_std20,atr,std_20
0,3.190599,3.666667,1.154701
1,3.190599,3.666667,1.154701
2,3.190599,3.666667,1.154701
3,3.335000,3.556667,1.000000
4,1.391118,3.702967,2.081666
5,0.752679,3.580988,2.309401
6,1.578893,3.719262,2.000000
